In [ ]:
import networkx as nx
import os
import numpy as np
import pandas as pd
from func_modules import ControllerGraph
from tabulate import tabulate
import matplotlib.pyplot as plt

## 1. Data

### Clean the dataframes

We read the datasets from the `csv` files, then we clean them according to the required steps in the `readme` file describing the homework:
- Space at the end and slash at the end are removed in the names from `hero_network`.
- Rows inducing loops are removed.
- Solve the Spider-Man problem.

In [ ]:
# Read the dataframes
edges = pd.read_csv(os.path.join('data', 'edges.csv'))
hero_network = pd.read_csv(os.path.join('data', 'hero-network.csv'))
nodes = pd.read_csv(os.path.join('data', 'nodes.csv'))

In [ ]:
# This simple Regex substitution solves slash and space problem in hero network
hero_network['hero1'] = hero_network.hero1.str.replace(r'/$|\s$', '', regex=True)
hero_network['hero2'] = hero_network.hero2.str.replace(r'/$|\s$', '', regex=True)

# Remove rows inducing loops in the graph
hero_network = hero_network[~(hero_network.hero1 == hero_network.hero2)].copy()

In [ ]:
# Fix Peter Parker name
hero_network.loc[hero_network.hero1 == 'SPIDER-MAN/PETER PAR', 'hero1'] = 'SPIDER-MAN/PETER PARKER'
hero_network.loc[hero_network.hero2 == 'SPIDER-MAN/PETER PAR', 'hero2'] = 'SPIDER-MAN/PETER PARKER'

### Build the graphs

Let's build the first graph, the one from `hero_network`, undirected and weighted, with weights decreasing as the number of common appearances in comics increases. The approach chosen is similar to an idf measure, with the weight being given by $\frac{1}{\log(x)+1}$.

In [ ]:
# Count appearances of hero 1 and hero 2 (ordered)
hero_counts = hero_network.value_counts()
# Group with frozenset, this way we get count of appearances without taking into consideration the order
hero_counts = hero_counts.groupby(lambda x: frozenset(x)).sum()
# Move frozenset from index to column
hero_counts = hero_counts.reset_index(name = 'count')

# List constructor onto frozenset (because pd.Series requires order) and then pd.Series to split the lists and get two different series
# (resulting in a DataFrame). Then concatenate count Series with these other two we have obtained. This is close to the final dataset we need.
hero_counts = pd.concat([hero_counts['index'].apply(list).apply(pd.Series), hero_counts['count']], axis = 1)

hero_counts['weight'] = 1/(1+np.log(hero_counts['count'])) # Compute weights with something quite similar to idf weights
hero_counts.drop('count', axis = 1, inplace = True) # Drop count column, we don't need that anymore
hero_counts.rename({0: 'hero1', 1:'hero2'}, axis = 1, inplace = True) # Rename for tidiness

# Build the undirected graph from edgelist + weight attribute
hero_graph = nx.from_pandas_edgelist(hero_counts, source='hero1', target='hero2', edge_attr='weight')

Now we build the second graph, the one from edges and considering the node type. For this we need to use `nodes` and `edges`.

In [ ]:
hero_comics_graph = nx.from_pandas_edgelist(edges, source='hero', target='comic')
nodes_attr_dict = nodes.set_index('node').to_dict(orient='index')
nx.set_node_attributes(hero_comics_graph, nodes_attr_dict)

We instantiate the controller object which is needed to call the functionalities.

In [ ]:
controller = ControllerGraph(edges) # Edges is passed for the top N heroes list

## Functionality 1: Explaining and Visualizing
### Heroes graph

Lorem ipsum

In [ ]:
number_of_nodes,number_of_collaborations,\
    heroes_in_comic,density,networks_degree_distribution,\
    average_degree,network_hubs,network = controller.functionality(1, hero_graph, graph_type= 1)

#### Table containing the following general information about the graph:

In [ ]:
table = [['Number of nodes in the network', number_of_nodes], ['Density of the network', density],['Average degree of the network',average_degree],["The network is", network]]
print(tabulate(table, tablefmt='grid',numalign="right"))

#### Table that lists the network's hubs

In [ ]:
table=list(network_hubs)
tab=[[i,table[i]] for i in range(len(table))]
#tab
print(tabulate(tab,tablefmt='github',headers=["ID","Network's hubs"]))

#### Plot depicting the number of collaborations of each hero in descending order
Lorem ipsum

In [ ]:
top_50=dict(number_of_collaborations.most_common(50))
plt.gcf().set_size_inches((12, 8))
plt.tick_params(axis='x', labelrotation=90)
plt.title("Number of collaborations of each hero")
plt.xlabel("Heroes")
plt.ylabel("Frequency")
plt.bar(top_50.keys(),top_50.values())
plt.show()

#### Plot depicting the degree distribution of the network

In [ ]:
plt.gcf().set_size_inches((12, 8))
plt.title("The network's degree distribution")
plt.plot(networks_degree_distribution)
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.show()

### Heroes and comics

In [ ]:
number_of_nodes,number_of_collaborations,\
    heroes_in_comic,density,networks_degree_distribution,\
    average_degree,network_hubs,average_clustering = controller.functionality(1, hero_comics_graph, graph_type= 2)

#### Table containing the following general information about the graph:

In [ ]:
table = [['Number of nodes in the network', number_of_nodes], ['Density of the network', density],['Average degree of the network',average_degree],["Average clustering coefficient",average_clustering]]
print(tabulate(table, tablefmt='grid',numalign="right"))

#### Table that lists the network's hubs

In [ ]:
table=list(network_hubs)
tab=[[i,table[i]] for i in range(len(table))]
#tab
print(tabulate(tab,tablefmt='github',headers=["ID","Network's hubs"]))

#### Plot depicting the number of heroes who appeared in each comic, sorted in descending order
Lorem ipsum


In [ ]:
top_50 = dict(sorted(heroes_in_comic.items(), key = lambda x: x[1], reverse = True)[:51])
plt.gcf().set_size_inches((12, 8))
plt.tick_params(axis='x', labelrotation=90)
plt.title("Number of heroes who appeared in each comic")
plt.xlabel("Comics")
plt.ylabel("Frequency")
plt.bar(top_50.keys(),top_50.values())
plt.show()

#### Plot depicting the degree distribution of the network

In [ ]:
plt.gcf().set_size_inches((12, 8))
plt.title("The network's degree distribution")
plt.plot(networks_degree_distribution)
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.show()

## Functionality 2: explaining and visualizing

We call `func2`, which you can inspect in the `functionality_2.py` module in the `func_modules` package, to take the values of the matric chosen by the user on the entire graph (or subgraph if the user chooses the number N of `top_heroes`) and on the hero selected. To take this values we use some functions imported by `networkx` like `nx.betweenness_centrality()`,  `nx.pagerank_numpy()`, `nx.closeness_centrality()` and `nx.degree_centrality()`.
Then we calculate the average of the requested centrality measure for all of the network's nodes.

In [ ]:
#The metric values calculated on the graph and on the selected hero
metric_values, hero_metric_value = controller.functionality(2,hero_graph, hero = 'CAPTAIN AMERICA', metric = 'betweenness_centrality')

In [ ]:
mean_centrality = np.array(list(metric_values.values())).mean() #The average of the requested centrality measure for all of the network's nodes

In [ ]:
table = [['Avarage of the centrality measure', mean_centrality], ['Centrality measure for the given hero', hero_metric_value]]
print(tabulate(table, tablefmt='grid'))

- The ***betweenness centrality*** of a node measures the extent to which the node lies on the shortest paths between other nodes. A high betweenness centrality value means that the node is located on many shortest paths and, therefore, has a *central position* within the graph. On the other hand, a low betweenness centrality value indicates that the node is not on many shortest paths and, therefore, has a *less central position* within the graph.

- The ***PageRank*** of a node measures the importance of the node within the graph, with higher values indicating more important nodes. A high PageRank value means that the node is more likely to be visited by a random walker and, therefore, has a *more central position* within the graph. On the other hand, a low PageRank value indicates that the node is less likely to be visited and, therefore, has a *less central position* within the graph.

- The ***closeness centrality*** of a node measures the average distance from the node to all other nodes in the graph. A high closeness centrality value means that the node is close to many other nodes and, therefore, has a *central position* within the graph. On the other hand, a low closeness centrality value indicates that the node is far from many other nodes and, therefore, has a *less central position* within the graph.

- The ***degree centrality*** of a node measures the number of edges incident to the node. A high degree centrality value means that the node has many connections to other nodes and, therefore, has a *central position* within the graph. On the other hand, a low degree centrality value indicates that the node has few connections to other nodes and, therefore, has a *less central position* within the graph.

If a user has a high *betweenness centrality* value, this means that they are located on many shortest paths within the graph and are therefore considered to be a key connector. If the user has a low *betweenness centrality* value but a high *PageRank* value, this may mean that they are not on many shortest paths but are still considered to be an important node within the graph due to the high number of visits they receive. Similarly, if the user has a high *closeness centrality* value, this means that they are close to many other nodes in the graph and are therefore considered to be a central node. If the user has a high *degree centrality* value, this means that they have many connections to other nodes and are therefore considered to be a central node within the graph.

## Functionality 3: explaining and visualizing

For the third functionality we are asked to build a function which, given the bipartite graph with heroes and comics, returns the shortest walk passing through a given set of heroes in order (the order of first visit is what matters), with the endpoints of the walk also given.

The straightforward implementation we can think of here is simply repeatedly using Breadth-First Search to get from one node to the next one in the sequence, exploiting the fact that we [know that](http://aris.me/contents/teaching/data-mining-ds-2019/resources/CLRS-graphs,bfs,dfs.pdf) exploring with BFS we get the shortest path. The ordered sequence of shortest paths is bound to be the shortest walk.

Therefore, `func_3`, which you can inspect in the `functionalities_34.py` module in the `func_modules` package, is simply a manual implementation of Breadth-First Search with a few tweaks. More specifically:

- As we have already said, BFS is the core of the functionality, but not the functionality as a whole. BFS is repeated, with the current target then becoming the source of the following iteration. The iteration starts with the first endpoint as the first source node and ends with the second endpoint as the last target node.
- Of course, we don't just need to explore the nodes reachable from a source, but we also need to store the path from the source to the visited node, each time. This is actually quite simple to do, and it can be done by simply storing in the BFS queue not only the identifier of the visited node, but also the path with which we can reach it from the source. This allows to have some _persistent memory_ in the process as more levels are explored. When the current target is reached, the `walk` list is extended with the shortest path found in the current iteration with BFS.
- We need to take into consideration the order of first visit. This simply amounts to hardcode the fact that the BFS should not visit nodes which follow the target one (for the current iteration) in the sequence.
- If at any iteration of our _repeated BFS_ the current target node cannot be reached with BFS, then the whole execution has to halt, print a proper warning message and return an empty list (the empty walk). This can be done by adding a boolean flag which is set to True only when the current target node is found at the current iteration.

Now we call the functionality through the `functionality` method of the controller, considering the whole graph. `seq` is the sequence of heroes we have to pass through, `endpoints` is the tuple giving the starting and terminal endpoints.

In [ ]:
shortest_walk = controller.functionality(3, hero_comics_graph,
                         seq = ['ABOMINATION/EMIL BLO', 'ANCIENT ONE', 'IRON MAN/TONY STARK', 'SPIDER-MAN/PETER PARKER'],
                         endpoints=('CAPTAIN AMERICA', 'CAPTAIN MARVEL/CAPTA'))

Now we visualize the result in a proper way. We make different calls to the NetworkX plotting functions (which use the matplotlib engine and are thus compatible with the usual framework) in order to build a complex visualization. For the plot we consider only the nodes and the edges in the walk, with the edges between them not present in the walk which are anyway drawn, but with a very low value of the `alpha` channel and with limited width.

As required, labels are added to the edges in order to highlight the ordering, which is anyway clear even without those. Since the graph is at this point planar, we can have a relatively clear visualization with the `planar_layout` function for the dictionary of positions. The hero nodes are blue and the comic nodes are red, and this difference in color highlights the fact that the graph is bipartite (a red always follows a blue, and vice-versa). It was also requested to print the comic nodes in the shortest walk: those are printed in the upper left corner of the plot.

In [ ]:
tuples_walk = list()
dict_labels_edge = dict()
for idx in range(len(shortest_walk)-1):  # Build list of edges in the walk and build dictionary of order indexes for edges
    tuples_walk.append((shortest_walk[idx], shortest_walk[idx+1]))
    dict_labels_edge[(shortest_walk[idx], shortest_walk[idx+1])] = idx+1

# Get subgraph to plot
plot_graph = nx.subgraph_view(hero_comics_graph, filter_node=lambda x: x in shortest_walk,
                              filter_edge = lambda x, y: (x, y) in tuples_walk or (y, x) in tuples_walk)
plt.gcf().set_size_inches((12, 10)) # Change size of the plot

dict_pos = nx.planar_layout(plot_graph) # Dictionary of positions
nx.draw_networkx_edges(plot_graph, pos = dict_pos, width=5)  # Draw edges in the walk

# Draw nodes with specific color, with labels (the names)
nx.draw_networkx_nodes(plot_graph, pos = dict_pos,
                       node_color = ['blue' if plot_graph.nodes[x]['type'] == 'hero' else 'red' for x in plot_graph.nodes])
nx.draw_networkx_labels(plot_graph, pos = dict_pos, horizontalalignment='left', verticalalignment='bottom')

# Draw edges with specific labels (the index highlighting the ordering)
nx.draw_networkx_edges(nx.subgraph_view(hero_comics_graph, filter_node=lambda x: x in shortest_walk),
                       pos = dict_pos, alpha = 0.1)
nx.draw_networkx_edge_labels(plot_graph, pos = dict_pos, edge_labels=dict_labels_edge, font_size=10)

plt.margins(x = 0.09)  # Add some margin in order not to cut labels
plt.text(-1.2, 0.45, s = 'Comics in shortest walk (red nodes): '+
                         ', '.join([x for x in shortest_walk if hero_comics_graph.nodes[x]['type'] == 'comic']))
plt.suptitle('Getting from Cap. America to Cap. Marvel saying \'Hi\' to some heroes', size = 20)
plt.title('Notice that a comic (red nodes) always follows a hero (blue nodes), and viceversa. \n'
          'This is due to the fact that the graph with heroes and comics is bipartite.', pad = 20)

plt.show()

## Functionality 4: Explaining and Visualizing

The main idea here is exploiting the [max-flow min-cut theorem](https://en.wikipedia.org/wiki/Max-flow_min-cut_theorem) which allow us to link max-flow problems to min-cut problems. In other words we have to disconnect the source from the sink inducing a cut/a partition, and we need to do that with the minimum cost/cumulative weight possible, thus finding the s-t cut with the minimum capacity over the set of all the possible ones.

Doing that is not trivial and requires some relatively advanced tools. First of all we need to build the residual graph, i.e. the graph representing the flow in the maximized scenario. In the original graph, the weight of an edge is its capacity, i.e. the maximum amount of flow it can carry. With the residual graph, we have a **directed** graph, since the flow has a direction: the residual capacity of each edge is given by $r_{i,j} = c_{i,j}-f_{i,j}$, with $c_{i,j}$ as the original capacity/the weight of the undirected edge between $i$ and $j$ and $f_{i,j}$ being the **directed** flow passing through the edge in the specific direction, which is negative if the flow goes backwards. It is trivial to notice, but I will stress it: $r_{i, j} \neq r_{j,i}$. To give an example, $r_{i, j} = 0 \rightarrow r_{j,i} \geq c_{j,i} = c_{i, j}$

For each original undirected edge we thus have two directed edges. Now, suppose that we can reach a node from the source only using edges with non-null residual capacity: we will not be able to reach the sink (otherwise we would have an augmenting path) and the nodes we reach are the ones which represent one of the two subsets induced by the minimum s-t cut, i.e. the one with minimum cost/capacity disconnecting source and sink. The edges we need to get a cut-set are the ones connecting these reachable nodes to the non-reachable ones.

This is exactly what we are doing in our implementation:
- We use [Edmonds-Karp](https://en.wikipedia.org/wiki/Edmonds–Karp_algorithm) (the [Ford-Fulkerson algorithm](https://en.wikipedia.org/wiki/Ford–Fulkerson_algorithm)) from NetworkX set of algorithms. This allows us to have the residual graph returned.
- Once we have that, we perform Breadth-First Search from the source using only the edges with non-null residual capacity. The nodes we reach are the ones in the first subset induced by the minimum s-t cut.
- Once we have the labels/the name of the nodes of the first subset, we simply cut each connection in the original graph with the others in the second. For that, we can use the `edge_boundary` function from NetworkX.

In [ ]:
num_edges_lost, edges_lost, cut_graph = controller.functionality(4, hero_graph, 35,
                                                                 hero_2='SPIDER-MAN/PETER PARKER', hero_1='IRON MAN/TONY STARK')

Now we visualize the output of the functionality: notice that we are considering only the top 35 heroes since otherwise it would be impossible to see anything clearly. The problem is that by taking the top 35 heroes the resulting graph of heroes (we are choosing that one since it is weighted) is complete: this means that in general the cut-set will always contain only edges adjacent to the source or the sink. It is just simple logic, jumping to a neighbour of source or sink for the cut produces a combinatorial explosion: instead of removing one edge in order to induce the cut from source to sink, we now have to remove $N-2$, with $N$ as the number of nodes. Apart from very strange distributions of the weights, this kind of situation induces s-t cut where the cut-set is adjacent to source or sink. The result is a cut $C(S, T)$ where $S = \{s\}$ or with $T = \{t\}$, with $s$ being the source and $t$ being the sink.

Notice that we would have an identical situation also increasing the number of nodes (increasing the top heroes number). Considering the graph without filtering for top heroes allows for the smaller cut to have more than one vertex, but the idea remains the same.

In order to visualize the min-cut s-t for the 35 heroes we chose not to plot two graphs - with and without the edges in the cut-set-, but only one highlighting the edges in the cut-set, which combines the two perspectives. The edges in the cut-set are shown in red, which is also the colour of source and sink. The rest of the edges are anyway shown, but with a lower alpha channel, which is anyway higher - relatively speaking - as the weight increases.

In [ ]:
plt.gcf().set_size_inches((12, 10)) # Increase size of plot
plot_graph = cut_graph.copy() # Copy graph (without the edges in the cut-set) to avoid modifying in place
plot_graph.add_edges_from(edges_lost) # Add edges which were lost (the one in the cut-set)

dict_pos = nx.circular_layout(plot_graph) # Circular layout because it is more clear
# Color source and sink
color_node = ['red' if node == 'SPIDER-MAN/PETER PARKER' or node == 'IRON MAN/TONY STARK' else 'blue' for node in plot_graph.nodes]
nx.draw_networkx_nodes(plot_graph, pos = dict_pos, node_size = 100, node_color=color_node) # Draw nodes
# Draw labels
nx.draw_networkx_labels(plot_graph, pos = dict_pos, font_size=6, font_family='helvetica', verticalalignment='bottom', font_weight='heavy')

# Set color list, red for edges in cut-set, gray otherwise
color_edges = ['red' if (edge in edges_lost or tuple(reversed(edge)) in edges_lost) else 'gray' for edge in plot_graph.edges]
# Set alpha channel, a function of the weight, 1 for edges in the cut-set (which do not have weight attribute)
alpha_edges = [x[2]/1.5 if x[2] else 1 for x in plot_graph.edges(data='weight')]
# Draw the edges with the alpha and color info we have built
nx.draw_networkx_edges(plot_graph, pos = dict_pos, alpha = alpha_edges, edge_color = color_edges)

# Add some margins
plt.margins(x=0.10)
plt.suptitle('Minimum effort, maximum result: disconnecting Spider-Man from Iron Man', size = 20)
plt.title(f'Removing the edges (n = {num_edges_lost}) which the sink is incident with induces the st-cut. \n'
          'This makes sense due to the fact that the graph is complete when considering\n'
          ' the top N heroes and anyway very dense in general.', pad = 12)
plt.show()